In [18]:
from typing import List, Optional
import json

from pydantic import BaseModel
from groq import Groq

from dotenv import load_dotenv
load_dotenv()

groq = Groq()


# Data model for LLM to generate
class Ingredient(BaseModel):
    name: str
    quantity: str
    quantity_unit: Optional[str]


class Recipe(BaseModel):
    recipe_name: str
    ingredients: List[Ingredient]
    directions: List[str]


def get_recipe(recipe_name: str) -> Recipe:
    chat_completion = groq.chat.completions.create(
        messages=[
            {
                "role": "system",
                "content": "You are a recipe database that outputs recipes in JSON.\n"
                # Pass the json schema to the model. Pretty printing improves results.
                f" The JSON object must use the schema: {json.dumps(Recipe.model_json_schema(), indent=2)}",
            },
            {
                "role": "user",
                "content": f"Fetch a recipe for {recipe_name}",
            },
        ],
        model="meta-llama/llama-4-scout-17b-16e-instruct",
        temperature=0,
        # Streaming is not supported in JSON mode
        stream=False,
        # Enable JSON mode by setting the response format
        response_format={"type": "json_object"},
    )
    return Recipe.model_validate_json(chat_completion.choices[0].message.content)


# def main():
recipe_name = "chocolate chip cookies"
recipe = get_recipe(recipe_name)
recipe

Recipe(recipe_name='Classic Chocolate Chip Cookies', ingredients=[Ingredient(name='All-purpose flour', quantity='2', quantity_unit='cups'), Ingredient(name='Baking soda', quantity='1', quantity_unit='teaspoon'), Ingredient(name='Salt', quantity='1', quantity_unit='teaspoon'), Ingredient(name='Granulated sugar', quantity='3/4', quantity_unit='cup'), Ingredient(name='Brown sugar', quantity='3/4', quantity_unit='cup'), Ingredient(name='Butter', quantity='1', quantity_unit='cup'), Ingredient(name='Large eggs', quantity='2', quantity_unit=None), Ingredient(name='Vanilla extract', quantity='1', quantity_unit='teaspoon'), Ingredient(name='Semi-sweet chocolate chips', quantity='2', quantity_unit='cups')], directions=['Preheat oven to 375°F (190°C). Line a baking sheet with parchment paper.', 'Whisk together flour, baking soda, and salt. Set aside.', 'Cream together sugars and butter until light and fluffy.', 'Beat in eggs one at a time, followed by vanilla extract.', 'Gradually mix in the dry 

In [19]:
recipe.model_dump()

{'recipe_name': 'Classic Chocolate Chip Cookies',
 'ingredients': [{'name': 'All-purpose flour',
   'quantity': '2',
   'quantity_unit': 'cups'},
  {'name': 'Baking soda', 'quantity': '1', 'quantity_unit': 'teaspoon'},
  {'name': 'Salt', 'quantity': '1', 'quantity_unit': 'teaspoon'},
  {'name': 'Granulated sugar', 'quantity': '3/4', 'quantity_unit': 'cup'},
  {'name': 'Brown sugar', 'quantity': '3/4', 'quantity_unit': 'cup'},
  {'name': 'Butter', 'quantity': '1', 'quantity_unit': 'cup'},
  {'name': 'Large eggs', 'quantity': '2', 'quantity_unit': None},
  {'name': 'Vanilla extract', 'quantity': '1', 'quantity_unit': 'teaspoon'},
  {'name': 'Semi-sweet chocolate chips',
   'quantity': '2',
   'quantity_unit': 'cups'}],
 'directions': ['Preheat oven to 375°F (190°C). Line a baking sheet with parchment paper.',
  'Whisk together flour, baking soda, and salt. Set aside.',
  'Cream together sugars and butter until light and fluffy.',
  'Beat in eggs one at a time, followed by vanilla extract